# CarND-Behavioral-Cloning-P3

## Read from csv file

In [13]:
import csv
import cv2
import numpy as np

def read_data(path):
    print("Start read training data from path: " + path)
    driving_log_path = path + "/driving_log.csv"
    lines = []
    
    with open(driving_log_path) as csv_file:
        reader = csv.reader(csv_file)
        for line in reader:
            lines.append(line)
    print("Read from csv file done!")
    
    images = []
    measurements = []
    for i, line in enumerate(lines):
        if i == 0:
            continue
        # read in center images
        image_path = line[0]
        file_name = image_path.split("/")[-1]
        file_path = path + "/IMG/"
        data_path = file_path + file_name
        image = cv2.imread(data_path)
        images.append(image)
        # read in angles
        measurement = float(line[3])
        measurements.append(measurement)
    print("Extract center images and angle measurements from file done! Get {0} samples".format(len(images)))
    
    # Data augment
    augment_images, augment_measurements = [], []
    for image, measurement in zip(images, measurements):
        augment_images.append(image)
        augment_measurements.append(measurement)
        augment_images.append(cv2.flip(image, 1))
        augment_measurements.append(measurement * -1.0)
    print("Data augment done!")
    print()

    return augment_images, augment_measurements
    

def merge_data(path, X_train = None, y_train = None):
    custom_images, custom_measurements = read_data(path)
    if X_train is None:
        print("Create data sets...")
        X_train = np.array(custom_images)
        y_train = np.array(custom_measurements)
    else:   
        print("Concat data sets...")
        X_train = np.concatenate((X_train, np.array(custom_images)))
        y_train = np.concatenate((y_train, np.array(custom_measurements)))
    return X_train, y_train

# Read from data/
X_train, y_train = merge_data("data")
X_train, y_train = merge_data("custom_data_0212", X_train, y_train)
X_train, y_train = merge_data("custom_data_0301", X_train, y_train)


print("Done! Get {0} samples.".format(len(X_train)))

Start read training data from path: data
Read from csv file done!


Extract center images and angle measurements from file done! Get 8035 samples


Data augment done!

Create data sets...


Start read training data from path: custom_data_0212
Read from csv file done!


Extract center images and angle measurements from file done! Get 7371 samples


Data augment done!

Concat data sets...


Start read training data from path: custom_data_0301
Read from csv file done!


Extract center images and angle measurements from file done! Get 1706 samples


Data augment done!

Concat data sets...


Done! Get 34224 samples.


### Test the training data

In [16]:
print("X_train shape: " + str(X_train.shape))
print("y_train shape: " + str(y_train.shape))

## Train the model

In [20]:
# # Train model
# from keras.models import Sequential, Model
# from keras.layers import Lambda, Flatten, Dense
# 
# print("Start training the model...")
# 
# model = Sequential()
# model.add(Flatten(input_shape=(160,320,3)))
# model.add(Dense(1))
# 
# model.compile(loss='mse', optimizer='adam')
# model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=8)
# 
# model.save("model.h5")
# print("Training done, save model to model.h5")

Turns out the flow is working. But it is not working fine. To improve the model we can do following things to improve the model:
1. Normalize the input images.
2. Crop the input image.
3. Use the Navida architecture.
4. Add dropout to the training architecture.

In [15]:
# Train model
import os  

from keras.models import Sequential, Model
from keras.layers import Lambda, Flatten, Dense, Cropping2D, Convolution2D, Dropout

print("Start training the model...")

model = Sequential()
model.add(Lambda(lambda x : (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.9))
model.add(Dense(50))
model.add(Dropout(0.9))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=3)

model_name = "models/model_test.h5"
model.save(model_name)
print("Training done, save model to {0}".format(model_name))

Start training the model...


Train on 27379 samples, validate on 6845 samples
Epoch 1/3


   32/27379 [..............................] - ETA: 1505s - loss: 0.0090

   64/27379 [..............................] - ETA: 982s - loss: 0.0403 

   96/27379 [..............................] - ETA: 794s - loss: 0.0350

...

27296/27379 [============================>.] - ETA: 0s - loss: 0.0061

27328/27379 [============================>.] - ETA: 0s - loss: 0.0062

27360/27379 [============================>.] - ETA: 0s - loss: 0.0061

27379/27379 [==============================] - 346s - loss: 0.0061 - val_loss: 0.1174

Training done, save model to models/model_0301_3.h5


Since the data is getting more and more. We have to change the read csv function to stream way to reduce memory consume.

In [90]:
import os
import csv
from itertools import chain

# Return generators of training and validation set
def pop_generator(path):

    samples = []
    with open(path + '/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            samples.append(line)

    from sklearn.model_selection import train_test_split
    train_samples, validation_samples = train_test_split(samples, test_size=0.2)

    import cv2
    import numpy as np
    import sklearn

    def generator(samples, batch_size=32):
        num_samples = len(samples)
        while 1: # Loop forever so the generator never terminates
            sklearn.utils.shuffle(samples)
            for offset in range(0, num_samples, batch_size):
                batch_samples = samples[offset:offset+batch_size]

                images = []
                angles = []
                for batch_sample in batch_samples:
                    name = path + '/IMG/'+batch_sample[0].split('/')[-1]
                    left_name = path + '/IMG/'+batch_sample[1].split('/')[-1]
                    right_name = path + '/IMG/'+batch_sample[2].split('/')[-1]
                    center_image = cv2.imread(name)
                    left_image = cv2.imread(left_name)
                    right_image = cv2.imread(right_name)
                    center_angle = 0.0

                    try:
                        center_angle = float(batch_sample[3])
                    except ValueError:
                        print("Skip head line...content: {0}".format(batch_sample[3]))
                        continue

                    # This is the param that influence the self correction behavior
                    correction = 0.4
                    left_angle = float(batch_sample[3]) + correction
                    right_angle = float(batch_sample[3]) - correction

                    images.append(center_image)
                    images.append(left_image)
                    images.append(right_image)
                    angles.append(center_angle)
                    angles.append(left_angle)
                    angles.append(right_angle)

                # Data augmentation
                augment_images, augment_angles = [], []
                for image, angle in zip(images, angles):
                    augment_images.append(image)
                    augment_angles.append(angle)
                    augment_images.append(cv2.flip(image, 1))
                    augment_angles.append(angle * -1.0)

                # trim image to only see section with road
                X_train = np.array(augment_images)
                y_train = np.array(augment_angles)
                yield sklearn.utils.shuffle(X_train, y_train)

    # compile and train the model using the generator function
    train_generator = generator(train_samples, batch_size=32)
    validation_generator = generator(validation_samples, batch_size=32)
    
    return train_generator, validation_generator, len(train_samples), len(validation_samples)


def pop_generators(dir_list):
    train_generator_list = []
    validation_generator_list = []
    train_count = 0
    validation_count = 0
    
    for dir_name in  dir_list:
        train_generator_tmp, validation_generator_tmp, train_count_tmp, validation_count_tmp = pop_generator(dir_name)
        train_generator_list.append(train_generator_tmp)
        validation_generator_list.append(validation_generator_tmp)
        train_count += train_count_tmp
        validation_count += validation_count_tmp

        
    train_generator = chain(train_generator_list[0], train_generator_list[1])
    for i in range(2, len(train_generator_list)):
        train_generator = chain(train_generator, train_generator_list[i])
        
    validation_generator = chain(validation_generator_list[0], validation_generator_list[1])
    for i in range(2, len(validation_generator_list)):
        validation_generator = chain(validation_generator, validation_generator_list[i])
        
    return train_generator, validation_generator, train_count - 1, validation_count - 1 # 减去一行表头


train_generator, validation_generator, train_count, validation_count = pop_generators(['data', 'custom_data_0301', 'custom_data_0302'])

print("train count: {0}, validation count: {1}".format(train_count, validation_count))

In [ ]:
# Train model

from keras.models import Sequential, Model
from keras.layers import Lambda, Flatten, Dense, Cropping2D, Convolution2D, Dropout

print("Start training the model...")

model = Sequential()
model.add(Lambda(lambda x : (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.9))
model.add(Dense(50))
model.add(Dropout(0.9))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator
                    , samples_per_epoch= ((train_count // 32 - 1) * 32) * 6
                    , validation_data=validation_generator
                    , nb_val_samples=((validation_count // 32 - 1) * 32) * 6
                    , nb_epoch=2)


model_name = "models/model_final.h5"
model.save(model_name)
print("Training done, save model to {0}".format(model_name))

